In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from keras.models import Model
from keras import optimizers
from keras.models import Sequential
from keras.layers import LSTM, Input, Embedding, Lambda, Dense, concatenate, Flatten, Dropout
from keras.layers.normalization import BatchNormalization
import keras.backend as K
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import StratifiedKFold, KFold,cross_val_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import mean_squared_error
from math import sqrt
from xgboost import XGBRegressor
import lightgbm as lgb

import datetime
from time import time
import gc

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from gensim.models import KeyedVectors

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

import xgboost as xgb
import lightgbm as lgb

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# Read the data

In [ ]:
train = pd.read_csv('../input/train.csv', encoding="ISO-8859-1")
test = pd.read_csv('../input/test.csv', encoding="ISO-8859-1")
# prod = pd.read_csv('../input/home-depot-product-search-relevance/product_descriptions.csv')

print('train size', train.shape)
print('test size', test.shape)
# print('prod size', prod.shape)

In [ ]:
train_size = train.shape[0]
df_all = pd.concat((train, test), axis=0, ignore_index=True)
# df_all = pd.merge(df_all, prod, on='product_uid', how='left')
# df_all['product_title'] = (df_all['product_title'] + df_all['product_description'])
# df_all.drop(['product_description', 'product_uid', 'id'], axis=1, inplace=True)

In [ ]:
df_all.head()

# Split product_full_info and search_term to chars

In [ ]:
char_to_remove = [' ', '{', '}', '"', '(', ')', '.', ',', '&','[',']','`','_','\'', '~', '\\', '-' 
                  '\x80', '\x81', '\x84', '\x89', '\x8b', '\x90', '\x93', '\x95', '\x99', '\x9a', 
                  '\x9d', '\xa0', '¡', '¢', 'ª', 'À', 'Â', 'È', 'Ê', 'Ë', 'Ï', 'Ò', 'Û', 
                  'Ü', 'â', 'ã', 'å', 'è', '÷']

def lower_char(char):
    if char >= 'A' and char <= 'Z':
        return char.lower()
    else:
        return char

df_all['product_title'] = df_all['product_title'].apply(lambda sent: [lower_char(character) for character in sent if character not in char_to_remove])
df_all['search_term'] = df_all['search_term'].apply(lambda sent: [lower_char(character) for character in sent if character not in char_to_remove])

# df_all['product_title'] = df_all['product_title'].apply(lambda x: x[:400])

In [ ]:
df_all.head()

# Get all unique chars to preform lable encoder

In [ ]:
prod_all_sentences = df_all['product_title']
search_all_sentences = df_all['search_term']

all_unique_chars = np.unique(np.concatenate((prod_all_sentences.append(search_all_sentences).values), axis=None))
all_unique_chars = np.append(['<uniq>'], all_unique_chars)

In [ ]:
all_unique_chars

# Replace tokens in numbers using lable encoder

In [ ]:
def token_to_num(data, unique_valus):
    le = LabelEncoder()
    le.fit(unique_valus)
    data['product_title'] = data['product_title'].apply(lambda char_list: le.transform(char_list).astype(float)) 
    data['search_term'] = data['search_term'].apply(lambda char_list: le.transform(char_list).astype(float)) 
    return data

df_all = token_to_num(df_all, all_unique_chars)

In [ ]:
df_all.head()

# Check max product description and search term length to know how much padding needed

In [ ]:
def get_max_length(data):
    max_len = 0
    for i in range(0, len(data)):
        if len(data.iloc[i]) > max_len:
            max_len = len(data.iloc[i])
    return max_len

def get_avg_length(data):
    len_sum = 0
    for i in range(0, len(data)):
        len_sum += len(data.iloc[i])
    return int(len_sum / len(data))

# avg_length_prod = get_avg_length(df_all['product_title'])
# avg_length_search = get_avg_length(df_all['search_term'])

max_length_prod = get_max_length(df_all['product_title'])
max_length_search = get_max_length(df_all['search_term'])

In [ ]:
# print('avg_length_prod', avg_length_prod)
# print('avg_length_search', avg_length_search)

print('max_length_prod', max_length_prod)
print('max_length_search', max_length_search)

# Split back to train and test sets

In [ ]:
df_train = df_all.iloc[:train_size]
df_test = df_all.iloc[train_size:]
df_test.reset_index(inplace=True, drop=True)

y_data = df_train['relevance']

# transform relevance to labels

In [ ]:
le_labels = LabelEncoder()
le_labels.fit(y_data.unique())
y_data_labels = le_labels.transform(y_data)

# Split to two sides:
1. search_term
2. product_desctiption + title

In [ ]:
X_data = {'left': df_train['product_title'], 'right': df_train['search_term']}
X_test = {'left': df_test['product_title'], 'right': df_test['search_term']}

# Add zero padding to each char list in size of max_length

In [ ]:
for dataset in [X_data, X_test]:
    dataset['left'] = pad_sequences(dataset['left'], maxlen=max_length_prod)
    dataset['right'] = pad_sequences(dataset['right'], maxlen=max_length_search)

In [ ]:
def reshpe_data(data):
    data['left'] = np.reshape(data['left'], (data['left'].shape[0], data['left'].shape[1], 1))
    data['right'] = np.reshape(data['right'], (data['right'].shape[0], data['right'].shape[1], 1))  
    return data

X_data = reshpe_data(X_data)
X_test = reshpe_data(X_test)

In [ ]:
print('X_train left shape', X_data['left'].shape)
print('X_train right shape', X_data['right'].shape)
print('y_train shape', y_data.shape)

print('X_test left shape', X_test['left'].shape)
print('X_test right shape', X_test['right'].shape)

In [ ]:
def create_onehot(data):
    onehot_all = []
    for line in data:
        onehot_encoded = list()
        for char in line:
            onehot = [0 for _ in range(len(all_unique_chars))]
            onehot[char[0]] = 1
            onehot_encoded.append(onehot)
        onehot_all.append(onehot_encoded)
    return onehot_all

rt = create_onehot(X_data['left'])

In [ ]:
for line in X_data['left']:
    for char in line:
        print(char)
        print(char[0])
        break
    break

# Build the model

In [ ]:
def get_model():

    # input layer
    left_input = Input(shape=(max_length_prod, 1))
    right_input = Input(shape=(max_length_search, 1))

    # LSTM layer
    shared_lstm = LSTM(n_lstm_hidden)
    left_output = shared_lstm(left_input)
    right_output = shared_lstm(right_input)

    # concat two outputs
    concat = concatenate([left_output, right_output])
    
    dense_1 = Dense(50, activation="relu")(concat)
    #dropout_1 = Dropout(0.1)(dense_1)

    # add Dense layer to calculate the similarty between product title and search term
    output = Dense(1, activation="relu")(dense_1)

    # Pack it all up into a model
    siamese_model = Model([left_input, right_input], output)

    # colmpiling
    siamese_model.compile(loss='mse', optimizer='adam')
    return siamese_model

In [ ]:
def split_to_train_val(data_x, data_y, train_index, val_index):
    train_left = data_x['left'][train_index]
    train_right = data_x['right'][train_index]
    y_train = data_y[train_index]  
    val_left = data_x['left'][val_index]
    val_right = data_x['right'][val_index]
    y_val = data_y[val_index]
    
    return train_left, train_right, val_left, val_right, y_train, y_val

In [ ]:
# Plot loss
def plot_loss(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper right')
    plt.show()

In [ ]:
# Model variables
n_lstm_hidden = 25
batch_size = 256
n_epoch = 5

pred_test = np.zeros(X_test['left'].shape[0])
mean_rmse = 0

n_splits = 5
# kfold = KFold(n_splits=n_splits, shuffle=True, random_state=24)
kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=24)

for train_index, test_index in kfold.split(X_data['left'], y_data_labels):    
    X_train_left_kfold, X_train_right_kfold, X_val_left_kfold, X_val_right_kfold, y_train_kfold, y_val_kfold = split_to_train_val(X_data, y_data.values, train_index, test_index)
    
    siamese_model = get_model()
    history = siamese_model.fit([X_train_left_kfold, X_train_right_kfold], y_train_kfold,
                            batch_size=batch_size, epochs=n_epoch,
                            validation_data=([X_val_left_kfold, X_val_right_kfold], y_val_kfold))
    
    pred_val = siamese_model.predict([X_val_left_kfold, X_val_right_kfold])
    rmse = sqrt(mean_squared_error(y_val_kfold, pred_val))
    mean_rmse += (rmse / n_splits)
    
    pred_test += np.reshape((siamese_model.predict([X_test['left'], X_test['right']]) / n_splits), (len(pred_test),))    
    
    print('rmse', rmse)
    plot_loss(history)
    
print('mean rmse', mean_rmse)

# Use the model we got as feature extractor 
1. for XGBoost model
2. for lightgbm model

In [ ]:
# Train the model on all the data
siamese_model = get_model()
history = siamese_model.fit([X_data['left'], X_data['right']], y_data.values, batch_size=batch_size, epochs=n_epoch)

# get the output of the concat layer and use it as features to the ml models
concat_layer = siamese_model.layers[3].output
feature_model = Model(siamese_model.input, concat_layer)
feature_model.compile(loss='mse', optimizer='adam')
print(feature_model.summary())

# we use the output of the concat layer as fetures so they will be the input to the xgb and lgb models
featurs = feature_model.predict([X_data['left'], X_data['right']])

# we preform the prediction also on the test set to evaluate the mse on test set
features_test = feature_model.predict([X_test['left'], X_test['right']])

In [ ]:
# # get the output of the concat layer and use it as features to the ml models
# concat_layer = siamese_model.layers[3].output
# feature_model = Model(siamese_model.input, concat_layer)
# feature_model.compile(loss='mse', optimizer='adam')
# print(feature_model.summary())

# # we use the output of the concat layer as fetures so they will be the input to the xgb and lgb models
# featurs = feature_model.predict([X_train_left_kfold, X_train_right_kfold])

# # we preform the prediction also on the test set to evaluate the mse on test set
# features_test = feature_model.predict([X_val_left_kfold, X_val_right_kfold])

In [ ]:
# xgb
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.01, gamma=0, subsample=0.8, colsample_bytree=1, max_depth=7)
xgb_model.fit(featurs, y_data)
xgb_pred = xgb_model.predict(features_test)

# lgb
lgb_model = lgb.sklearn.LGBMRegressor(is_unbalance=True, learning_rate =0.01, subsample=0.8, colsample_bytree=0.6, max_depth=7)
lgb_model.fit(featurs, y_data)
lgb_pred = lgb_model.predict(features_test)


# xgb_rmse = np.sqrt(mean_squared_error(y_val_kfold, xgb_pred))
# lgb_rmse = np.sqrt(mean_squared_error(y_val_kfold, lgb_pred))

In [ ]:
def round_pred(pred):
    for i in range(0,len(pred)):
        if pred[i] < 1:
            pred[i] = 1
        if pred[i] > 3:
            pred[i] = 3
    return pred

xgb_pred = round_pred(xgb_pred.astype(float))
lgb_pred = round_pred(lgb_predastype(float))

In [ ]:
# print(xgb_rmse, lgb_rmse)

# Write predictions to sample submission file

In [ ]:
sample_sub = pd.DataFrame()
sample_sub['id'] = test['id']
sample_sub['relevance'] = pred_test
sample_sub.to_csv('sample_submmision_char_siamese.csv', index=False)

sample_sub['relevance'] = xgb_pred
sample_sub.to_csv('sample_submmision_char_xgb.csv', index=False)

sample_sub['relevance'] = lgb_pred
sample_sub.to_csv('sample_submmision_char_lgb.csv', index=False)